In [1]:
import cv2
import urllib.request
from PIL import Image
from io import BytesIO

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def mk_matrix(img):
    ''' Convert PIL [img] into RGB and HSV Matrix,
    and put them into the [img] Object.
    '''
    img.matrix = dict()
        
    rgb = np.array(img.convert('RGB'))
    
    hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
    
    img.matrix['RGB'] = rgb
    img.matrix['HSV'] = hsv
    
    return img

In [3]:
def ravel(d):
    ''' Ravel the 1st and the 2nd Dimensions,
    only keep 500 Pixels to save Computation Time. 
    '''
    s = d.shape
    n = s[0] * s[1]
    d = d.reshape((n, s[2]))
    if n > 500:
        return d[range(0, n, int(n/500))]
    else:
        return d

def convert(rgb):
    ''' Convert RGB [rgb] values into #FFFFFF like Color String'''
    
    def n2s(n):
        s = hex(n).replace('x', '')
        return s[-2:]
    
    return '#' + ''.join([n2s(n) for n in rgb])

In [4]:
urls = [
#     'https://img-prod-cms-rt-microsoft-com.akamaized.net/cms/api/am/imageFileData/RE4wyTq?ver=c8e5',
#     'http://localhost:8000/DP164837.jpg',
#     'http://localhost:8000/132457451652521207_new.jpg',
#     'http://localhost:8000/DT86.jpg',
#     'http://localhost:8000/CubeChaos.png',
#     'http://localhost:8000/yosemite-5932x3337-5k-4k-wallpaper-8k-forest-osx-apple-mountains-181.jpg'
]

In [5]:
for url in urls:
    req = urllib.request.urlopen(url)
    im = Image.open(BytesIO(req.read()))
    im.matrix = dict()

    mk_matrix(im)

    v_rgb = ravel(im.matrix['RGB'])
    df = pd.DataFrame(ravel(im.matrix['RGB']), columns=['R', 'G', 'B'])
    df[['H', 'S', 'V']] = ravel(im.matrix['HSV'])
    df['color'] = [convert(e) for e in v_rgb]

    subplot_titles = ('RG', 'HS', 'GB', 'SV', 'RB', 'HV')
    subplot_pos = ((1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2))
    fig = make_subplots(rows=3, cols=2, subplot_titles=subplot_titles)

    for pos, title in zip(subplot_pos, subplot_titles):
        x, y = title
        row, col = pos

        _fig = px.scatter(df, x=x, y=y, title=title)
        _fig.data[0]['marker']['color'] = df['color']

        kwargs = dict(
            row=row,
            col=col
        )

        fig.add_trace(
            _fig.data[0],
            **kwargs
        )

        fig.update_xaxes(title_text=x, **kwargs)
        fig.update_yaxes(title_text=y, **kwargs)

    fig.update_layout(dict(
        height=1000,
        width=900,
        title=url
    ))
    fig.show()

    fig1 = px.imshow(im)
    fig1.show()